<a href="https://colab.research.google.com/github/NishitSingh2023/LSTM-in-Trade/blob/master/Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/Project/LSTM-in-Trade

! git pull

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import os
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from subprocess import check_output
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from sklearn.model_selection import  train_test_split
import time
import matplotlib.pyplot as plt
from numpy import newaxis

F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
F:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is depr

In [ ]:
df = pd.read_csv('DataSet/TATASTEEL.csv', usecols=['Date', 'Symbol', 'Open', 'High', 'Low', 'Close', 'Volume'])

In [ ]:
df.count()

In [ ]:
df.head()

In [ ]:
data = df.sort_index(ascending=True, axis=0)
new_data = pd.DataFrame(index=range(0,len(df)),columns=['Date', 'Close'])
for i in range(0,len(data)):
    new_data['Date'][i] = data['Date'][i]
    new_data['Close'][i] = data['Close'][i]

In [7]:
new_data.index = new_data.Date
new_data.drop('Date', axis=1, inplace=True)

#creating train and test sets
dataset = new_data.values

train = dataset[0:4238,:]
valid = dataset[4238:,:]

In [8]:
print(train)

[[152.45]
 [150.8]
 [156.55]
 ...
 [388.45]
 [392.35]
 [391.25]]


In [9]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

In [10]:
x_train, y_train = [], []
for i in range(60,len(train)):
    x_train.append(scaled_data[i-60:i,0])
    y_train.append(scaled_data[i,0])
x_train, y_train = np.array(x_train), np.array(y_train)

x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))

In [11]:
print(x_train)

[[[0.0924429 ]
  [0.09065263]
  [0.09689144]
  ...
  [0.03987414]
  [0.04242391]
  [0.0407964 ]]

 [[0.09065263]
  [0.09689144]
  [0.10958607]
  ...
  [0.04242391]
  [0.0407964 ]
  [0.04481094]]

 [[0.09689144]
  [0.10958607]
  [0.11360061]
  ...
  [0.0407964 ]
  [0.04481094]
  [0.05197201]]

 ...

 [[0.33309825]
  [0.34242934]
  [0.35143493]
  ...
  [0.35110942]
  [0.33944556]
  [0.35279119]]

 [[0.34242934]
  [0.35143493]
  [0.35094667]
  ...
  [0.33944556]
  [0.35279119]
  [0.3485054 ]]

 [[0.35143493]
  [0.35094667]
  [0.35002441]
  ...
  [0.35279119]
  [0.3485054 ]
  [0.35273694]]]


In [12]:
model = Sequential()
model.add(LSTM(180, return_sequences=True, input_shape = (x_train.shape[1],1)))
model.add(Dropout(0.2))

model.add(LSTM(180, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(200, return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(units=1))
model.add(Activation('relu'))

start = time.time()
model.compile(loss='mean_squared_error', optimizer='adam')
print ('compilation time : ', time.time() - start)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

compilation time :  0.12469744682312012


In [ ]:
#model.fit( epochs=10, batch_size=32, verbose=2)
model.fit(x_train, y_train, batch_size=10, epochs=10, verbose=2)

Epoch 1/10
